<a href="https://colab.research.google.com/github/sidhu2690/PyTorchNotes/blob/main/Fewshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
import torch.nn.functional as F
import torchvision.models as models

In [21]:
support = torch.rand(3, 224, 224)        # Support image [3(channels), H, W]

In [22]:
support

tensor([[[0.3670, 0.3849, 0.7128,  ..., 0.7659, 0.5533, 0.3993],
         [0.3849, 0.7625, 0.0222,  ..., 0.0487, 0.8327, 0.2431],
         [0.1262, 0.9219, 0.1035,  ..., 0.3526, 0.0662, 0.5358],
         ...,
         [0.2698, 0.8070, 0.2560,  ..., 0.0771, 0.4914, 0.1026],
         [0.4163, 0.1472, 0.6592,  ..., 0.1088, 0.3925, 0.7766],
         [0.7420, 0.0616, 0.3579,  ..., 0.0661, 0.2513, 0.3340]],

        [[0.4294, 0.3467, 0.1538,  ..., 0.3395, 0.3477, 0.4456],
         [0.0685, 0.5346, 0.6238,  ..., 0.4994, 0.2978, 0.1885],
         [0.1897, 0.1930, 0.2513,  ..., 0.4904, 0.1209, 0.9144],
         ...,
         [0.0246, 0.9686, 0.6264,  ..., 0.0577, 0.5235, 0.0133],
         [0.3263, 0.3558, 0.9170,  ..., 0.5186, 0.8129, 0.3116],
         [0.0984, 0.5470, 0.3411,  ..., 0.8207, 0.1965, 0.7161]],

        [[0.5125, 0.0393, 0.8017,  ..., 0.2089, 0.4658, 0.2983],
         [0.9459, 0.9630, 0.1058,  ..., 0.4171, 0.7516, 0.1078],
         [0.4974, 0.2583, 0.3418,  ..., 0.1859, 0.5374, 0.

In [34]:
support_mask = torch.randint(0, 2, (1, 224, 224))  # Binary mask [1, H, W]
support_mask.shape

torch.Size([1, 224, 224])

In [35]:
support_mask

tensor([[[1, 1, 0,  ..., 1, 1, 0],
         [1, 1, 1,  ..., 0, 0, 1],
         [0, 1, 1,  ..., 1, 1, 0],
         ...,
         [1, 0, 0,  ..., 1, 1, 1],
         [1, 0, 1,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 1, 1, 0]]])

In [26]:
query = torch.rand(3, 224, 224)          # Query image [3, H, W]
query

tensor([[[0.0430, 0.8777, 0.0042,  ..., 0.1638, 0.8323, 0.1562],
         [0.2690, 0.7867, 0.2250,  ..., 0.6713, 0.8562, 0.3493],
         [0.0455, 0.2147, 0.0109,  ..., 0.6151, 0.9197, 0.7255],
         ...,
         [0.4520, 0.5346, 0.4406,  ..., 0.7609, 0.2873, 0.4830],
         [0.3681, 0.9333, 0.7410,  ..., 0.4409, 0.0230, 0.2316],
         [0.2725, 0.9238, 0.0199,  ..., 0.2023, 0.2799, 0.4686]],

        [[0.6356, 0.1745, 0.2270,  ..., 0.8341, 0.8386, 0.6553],
         [0.2917, 0.2804, 0.7019,  ..., 0.5003, 0.9735, 0.2310],
         [0.8166, 0.9810, 0.9251,  ..., 0.0575, 0.8422, 0.5844],
         ...,
         [0.4433, 0.5148, 0.7845,  ..., 0.8362, 0.4182, 0.9804],
         [0.1572, 0.1343, 0.3693,  ..., 0.6557, 0.8095, 0.9231],
         [0.6006, 0.3899, 0.1949,  ..., 0.6105, 0.0997, 0.9996]],

        [[0.2525, 0.8038, 0.6728,  ..., 0.5831, 0.9798, 0.6096],
         [0.2751, 0.9170, 0.0024,  ..., 0.3646, 0.0625, 0.3939],
         [0.7845, 0.0049, 0.1088,  ..., 0.6697, 0.8564, 0.

In [27]:
query_mask = torch.randint(0, 2, (1, 224, 224))
query_mask

tensor([[[1, 0, 0,  ..., 0, 0, 1],
         [1, 0, 1,  ..., 1, 1, 1],
         [0, 1, 0,  ..., 0, 0, 0],
         ...,
         [1, 1, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0]]])

In [36]:
#Feature Extractor
resnet = models.resnet18(weights=None)
backbone = torch.nn.Sequential(*list(resnet.children())[:5])  # Output: [64, 56, 56]
backbone

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [ ]:
support_feat = backbone(support.unsqueeze(0))  # [1, 64, 56, 56]
query_feat = backbone(query.unsqueeze(0))      # [1, 64, 56, 56]

In [37]:
# Downsample the mask dimension to match feature map
support_mask_ds = F.interpolate(support_mask.unsqueeze(0).float(), size=(56, 56), mode='nearest')  # [1, 1, 56, 56]

In [38]:
#Prototype  (Coloured plane in feature space)
masked_feat = support_feat * support_mask_ds       # [1, 64, 56, 56]
proto = masked_feat.sum(dim=(2, 3)) / (support_mask_ds.sum() + 1e-5)  # [1, 64] prototype vector

In [ ]:
#Compute Cosine Similarity with Query Feature
q = query_feat.squeeze(0)         # [64, 56, 56]
q_flat = q.view(64, -1)           # [64, 3136]
q_norm = F.normalize(q_flat, dim=0)  # [64, 3136]
p_norm = F.normalize(proto.squeeze(0), dim=0).unsqueeze(1)  # [64, 1]
cos_sim = torch.matmul(p_norm.t(), q_norm).view(56, 56)     # [56, 56]

In [39]:
cos_sim.shape

torch.Size([56, 56])

In [64]:
cos_sim_upsampled = F.interpolate(cos_sim.unsqueeze(0).unsqueeze(0), size=(224, 224))

In [65]:
cos_sim_upsampled.shape

torch.Size([1, 1, 224, 224])

In [66]:
cos_sim_upsampled

tensor([[[[0.6996, 0.6996, 0.6996,  ..., 0.7142, 0.7142, 0.7142],
          [0.6996, 0.6996, 0.6996,  ..., 0.7142, 0.7142, 0.7142],
          [0.6996, 0.6996, 0.6996,  ..., 0.7142, 0.7142, 0.7142],
          ...,
          [0.6778, 0.6778, 0.6778,  ..., 0.7713, 0.7713, 0.7713],
          [0.6778, 0.6778, 0.6778,  ..., 0.7713, 0.7713, 0.7713],
          [0.6778, 0.6778, 0.6778,  ..., 0.7713, 0.7713, 0.7713]]]],
       grad_fn=<UpsampleNearest2DBackward0>)

#### mask prediction

In [67]:
# Decoder
decoder = torch.nn.Sequential(
    torch.nn.Conv2d(1, 16, 3, padding=1),  # [1, 224, 224] -> [16, 224, 224]
    torch.nn.ReLU(),
    torch.nn.Conv2d(16, 1, 1)              # [16, 224, 224] -> [1, 224, 224]
)

In [68]:
output = decoder(cos_sim_upsampled)        # Shape: [1, 1, 224, 224]
output_sigmoid = torch.sigmoid(output)     # [1, 1, 224, 224]
pred_mask = (output_sigmoid > 0.5).float() # Binary mask: [1, 1, 224, 224]

In [69]:
pred_mask

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]])